<a href="https://colab.research.google.com/github/atolia/colabs/blob/main/Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
try:
  import yfinance as yf
  import mibian
except Exception:
  !pip install yfinance > /dev/null 2>&1
  !pip install mibian >/dev/null
  import yfinance as yf
  import mibian
import pandas as pd
from google.colab import data_table
from datetime import datetime
import math
import numpy as np
# from types import SimpleNamespace
from IPython.display import Markdown, display, Javascript

def printmd(string):
    display(Markdown(string))

date_format = "%Y-%m-%d"
print('Please enter symbol name and click menu Runtime->Run all. This notebook fetch data from Yahoo finance. Options data usually available after 9:30AM EST')

#@title Stock common info { vertical-output: true, display-mode: "form" }
symbol = "NOBL" #@param {type:"string"}
ticker = yf.Ticker(symbol) 
distanceStrike =   6#@param {type:"integer"}
distanceDate =   6#@param {type:"integer"}
commissionUSD = 1.5 #@param {type:"number"}
hist1y = ticker.history(period='1y')
# ' '.join([prop for prop in ticker.info ])

def showPUTs(ticker, deltaLimit=-0.3 ):
  final = [];
  tdPrc = ticker.info['regularMarketPrice']
  ytdPrc = ticker.info['regularMarketPreviousClose']
  diff= (tdPrc-ytdPrc)/ytdPrc*100
  interest_rate = 0
  colsArr = [ 'days', 'Strike', 'today Change%', 'Profit $', '$/day', 'Profit %', 'Profit 1Y %', 'IV', 'OI', 'Delta', 'DualDelta', 'Gamma']
  for j in range(0, min(distanceDate, len(ticker.options)-1)):
    out = [];
    opdtstr = ticker.options[j]
    puts = ticker.option_chain(opdtstr).puts
    inMndx1 = puts.loc[puts['inTheMoney'] == True].index
    if len(inMndx1)>0:
      inMndx = inMndx1[0] 
    if inMndx < 1:
      continue
    opdt = datetime.strptime(opdtstr, date_format)
    # print( f'\n   ========== Expiration date: {opdtstr} ==================', end='')
    for i in range(max(0, (inMndx-distanceStrike-1)), inMndx):
      dysLeft = (opdt-datetime.now()).days+1
      if dysLeft < 2 or i == len(puts)-1:
        continue
      if puts.iloc[i].inTheMoney:
        continue
      prc = puts.iloc[i].bid or puts.iloc[i].lastPrice or puts.iloc[i].ask
      earnPerDay = round(prc/dysLeft*100, 2)
      OI = round(puts.iloc[i].openInterest, 2)
      IV = round(puts.iloc[i].impliedVolatility*100, 2)
      # p = mibian.BS(  [tdPrc, puts.iloc[i].strike, interest_rate, dysLeft], putPrice=prc);
      p = mibian.BS(  [tdPrc, puts.iloc[i].strike, interest_rate, dysLeft], volatility=IV);

      profitUSD = prc*100 - commissionUSD
      profit = profitUSD/tdPrc
      change = 0 if math.isnan(puts.iloc[i].percentChange) else round(puts.iloc[i].percentChange)
      toAdd = [ dysLeft, puts.iloc[i].strike, change, profitUSD, earnPerDay, profit, profit/dysLeft*365, IV, OI, p.putDelta, p.putDelta2, p.gamma]
      out.append(toAdd)
      final.append(toAdd)
    # display(pd.DataFrame(out, columns = colsArr ).round(2))

  # openInterest	impliedVolatility
  df = pd.DataFrame(final, columns = colsArr).sort_values(by=colsArr[6], ascending=False)
  df = df[df.Delta > deltaLimit]

  printmd(f" - **{ticker._info.get('symbol')}** Prev.Day price {round(ytdPrc,2)}, Current price **{round(tdPrc,2)}**, diff: {round(diff,2)}%.")
  yr = ticker.history(interval='1d', period='52wk')['Close']
  printmd(' - **1yr min:** ``$' + str(round((hist1y['Close'].min()), 2)) + '``' +
          ' **max:** ``$' + str(round((hist1y['Close'].max()), 2)) + '``' +
          ' **mean:** ``$' + str(round((hist1y['Close'].mean()), 2)) + '``' +
          ' **std:** ``$' + str(round((hist1y['Close'].std()), 2)) + '``' +
          ' **variance:** ' + str(round(hist1y['Close'].std()/hist1y['Close'].mean()*100, 2) ) + '%' 
          ' **Year by day Hist. Volatility:**'+ ' '+ str(round(yr.pct_change().std()*np.sqrt(365)*100,2))+'%')  
  printmd(f"Final list OutOfMoney PUTS for nearest {distanceStrike} Strikes and {distanceDate} Dates. Sorted by **1 year profit**. With Delta lower than **"+str(deltaLimit)+"**:")

  return df


if len(ticker.get_splits()):
  printmd('**Splits:** ')
  ticker.get_splits()




Please enter symbol name and click menu Runtime->Run all. This notebook fetch data from Yahoo finance. Options data usually available after 9:30AM EST


In [12]:
#@title Overal INFO { vertical-output: true }
if  ticker.info.get('longName'):
  printmd('- **Name:** ' + ticker.info['longName'] )

printmd('- **Expiration dates:** '+' '.join([dt for dt in ticker.options ]) )
printmd('- **Country:** ' + getattr(ticker,'country', 'Unknown') +' **Sector:** ' + getattr(ticker,'sector', 'Unknown') + ' **Industry:** ' + getattr(ticker,'industry', 'Unknown') + ' **MarketCap:** ' + str(round( getattr(ticker,'marketCap', 0) /1000/1000/1000,2)) + "B" )

if  ticker.info.get('longBusinessSummary'):
  printmd('- **Summary:** ' + ticker.info['longBusinessSummary'] )
if  ticker.info.get('dividendYield'):
  printmd('- **dividendYield:** ' + str(round((ticker.info['dividendYield'] or 0)*100, 2)) + '%' + 
          ' **trailingAnnualDividendYield:** ' + str(round((ticker.info['trailingAnnualDividendYield'] or 0)*100, 2)) + '%' +
          ' **Frwd.dividend/yr:** ' + str(round((ticker.info['dividendRate'] or 0), 2)) + '$' )
  printmd('- **exDividendDate:** ' + datetime.strftime(datetime.fromtimestamp(ticker.info['exDividendDate'] or 0), date_format+' %Z'))
else:
  print('No dividends info available ')  
printmd('- **totalRevenue:** ' + str(round(getattr(ticker,'totalRevenue', 0)/1e9, 2)) + 'B' + ' **revenueGrowth:** ' + str(round(getattr(ticker,'revenueGrowth', 0)*100, 2)) + '%' + ' **grossMargins:** ' + str(round(getattr(ticker,'grossMargins',0)*100, 2)) + '%' + ' **ebitdaMargins:** ' + str(round(getattr(ticker,'ebitdaMargins',0)*100, 2)) + '%')
if  ticker.info.get('trailingEps'):
  printmd('- **trailingEps:** ' + str(round(ticker.info['trailingEps'], 2)) + '\$' + ' **forwardEps:** ' + str(round(ticker.info['forwardEps'], 4)) + '$')
else:
  print('No EPS')  
yr=ticker.history(interval='1d', period='52wk')['Close']
printmd('- **StockPrice:** ``$' + str(round(ticker.info['regularMarketPrice'], 2)) + '``' +
        ' **1yr min:** ``$' + str(round((hist1y['Close'].min()), 2)) + '``' +
        ' **max:** ``$' + str(round((hist1y['Close'].max()), 2)) + '``' +
        ' **mean:** ``$' + str(round((hist1y['Close'].mean()), 2)) + '``' +
        ' **std:** ``$' + str(round((hist1y['Close'].std()), 2)) + '``' +
        ' **variance:** ' + str(round(hist1y['Close'].std()/hist1y['Close'].mean()*100, 2) ) + '%\n' +
        '-  **Hist. Volatil last year:** ' + str(round(yr.pct_change().std()*np.sqrt(365)*100,2))+'%'

        )
if len(ticker.get_splits()):
  printmd('**Splits:** ')
  ticker.get_splits()



- **Name:** ProShares S&P 500 Dividend Aristocrats ETF

- **Expiration dates:** 2022-05-20 2022-06-17 2022-07-15 2022-10-21

- **Country:** Unknown **Sector:** Unknown **Industry:** Unknown **MarketCap:** 0.0B

- **Summary:** The fund will invest at least 80% of its total assets in component securities of the index. The index contains a minimum of 40 stocks, which are equally weighted, and no single sector is allowed to comprise more than 30% of the index weight. It seeks to remain fully invested at all times in securities and/or financial instruments that, in combination, provide exposure to the returns of the index without regard to market conditions, trends or direction.

No dividends info available 


- **totalRevenue:** 0.0B **revenueGrowth:** 0% **grossMargins:** 0% **ebitdaMargins:** 0%

No EPS


- **StockPrice:** ``$89.29`` **1yr min:** ``$87.5`` **max:** ``$98.33`` **mean:** ``$92.82`` **std:** ``$2.46`` **variance:** 2.65%
-  **Hist. Volatil last year:** 16.74%

In [15]:
#@title Best Puts to sell
df = showPUTs(yf.Ticker(symbol), -0.4) 
data_table.DataTable(df.round(2), include_index=False, num_rows_per_page=200) # , min_width="500"

 - **NOBL** Prev.Day price 89.95, Current price **89.29**, diff: -0.73%.

 - **1yr min:** ``$87.5`` **max:** ``$98.33`` **mean:** ``$92.82`` **std:** ``$2.46`` **variance:** 2.65% **Year by day Hist. Volatility:** 16.74%

Final list OutOfMoney PUTS for nearest 6 Strikes and 6 Dates. Sorted by **1 year profit**. With Delta lower than **-0.4**:

,days,Strike,today Change%,Profit $,$/day,Profit %,Profit 1Y %,IV,OI,Delta,DualDelta,Gamma
0,64,85.0,0,185.5,2.92,2.08,11.85,3.13,0,-0.0,0.0,0.0
1,64,88.0,0,143.5,2.27,1.61,9.17,0.78,0,-0.0,0.0,0.0
2,64,89.0,0,133.5,2.11,1.50,8.53,0.20,0,-0.0,0.0,0.0


Dual Delta is like Delta but caclulated from Strike instead of StockPrice [More about Dual Delta ](http://jtoll.com/post/duel-of-the-deltas/)

In [16]:
#@title Best QQQ Puts to sell to compare

df = showPUTs(yf.Ticker('QQQ'), -0.4) 
data_table.DataTable(df.round(2), include_index=False, num_rows_per_page=200)

 - **QQQ** Prev.Day price 300.76, Current price **291.84**, diff: -2.97%.

 - **1yr min:** ``$87.5`` **max:** ``$98.33`` **mean:** ``$92.82`` **std:** ``$2.46`` **variance:** 2.65% **Year by day Hist. Volatility:** 28.35%

Final list OutOfMoney PUTS for nearest 6 Strikes and 6 Dates. Sorted by **1 year profit**. With Delta lower than **-0.4**:

,days,Strike,today Change%,Profit $,$/day,Profit %,Profit 1Y %,IV,OI,Delta,DualDelta,Gamma
6,4,291.0,0,465.5,116.75,1.60,145.55,0.78,0.0,-0.0,0.0,0.00
13,6,291.0,0,678.5,113.33,2.32,141.43,0.78,0.0,-0.0,0.0,0.02
5,4,290.0,0,424.5,106.50,1.45,132.73,1.56,0.0,-0.0,0.0,0.00
4,4,289.0,0,423.5,106.25,1.45,132.42,1.56,0.0,-0.0,0.0,0.00
12,6,290.0,0,559.5,93.50,1.92,116.63,1.56,0.0,-0.0,0.0,0.00
20,8,291.0,0,730.5,91.50,2.50,114.20,0.39,0.0,-0.0,0.0,0.00
10,6,288.0,0,542.5,90.67,1.86,113.08,3.13,0.0,-0.0,0.0,0.00
3,4,288.0,0,353.5,88.75,1.21,110.53,3.13,0.0,-0.0,0.0,0.00
11,6,289.0,0,527.5,88.17,1.81,109.96,1.56,0.0,-0.0,0.0,0.00
1,4,286.0,0,344.5,86.50,1.18,107.72,3.13,0.0,-0.0,0.0,0.00


In [7]:
#@title Best SPY Puts to sell to compare

df = showPUTs(yf.Ticker('SPY'), -0.4) 
data_table.DataTable(df.round(2), include_index=False, num_rows_per_page=200) 

 - **SPY** Prev.Day price 399.09, Current price **392.75**, diff: -1.59%.

- Final list OutOfMoney PUTS for nearest 6 Strikes and 6 Dates. Sorted by **1 year profit**. With Delta lower than **-0.4**

 - **1yr min:** ``$87.5`` **max:** ``$98.33`` **mean:** ``$92.82`` **std:** ``$2.46`` **variance:** 2.65% **Year by day Hist. Volatility:** 20.61%

,days,Strike,today Change%,Profit $,$/day,Profit %,Profit 1Y %,IV,OI,Delta,DualDelta,Gamma
6,4,392.0,0,505.5,126.75,1.29,117.45,0.39,0.0,-0.0,0.0,0.00
5,4,391.0,0,458.5,115.00,1.17,106.53,0.78,0.0,-0.0,0.0,0.00
13,6,392.0,0,629.5,105.17,1.60,97.50,0.39,0.0,-0.0,0.0,0.00
4,4,390.0,0,415.5,104.25,1.06,96.54,1.56,0.0,-0.0,0.0,0.00
3,4,389.0,0,373.5,93.75,0.95,86.78,1.56,0.0,-0.0,0.0,0.00
20,8,392.0,0,734.5,92.00,1.87,85.33,0.39,0.0,-0.0,0.0,0.01
12,6,390.0,0,549.5,91.83,1.40,85.11,1.56,0.0,-0.0,0.0,0.00
19,8,391.0,0,694.5,87.00,1.77,80.68,0.78,0.0,-0.0,0.0,0.00
2,4,388.0,0,345.5,86.75,0.88,80.27,3.13,0.0,-0.0,0.0,0.00
18,8,390.0,0,655.5,82.12,1.67,76.15,1.56,0.0,-0.0,0.0,0.00


Overal info:

In [8]:
#@title Best CALLS to sell

# symbol = "PLTR" 
# # symbol = "VIPS" 
# ticker = yf.Ticker(symbol) 
# distanceStrike =   5 
# distanceDate =   6 


final = []; 
tdPrc = ticker.info['regularMarketPrice']
ytdPrc = ticker.info['regularMarketPreviousClose']
diff= (tdPrc-ytdPrc)/ytdPrc*100
interest_rate = 0
colsArr = [ 'days', 'Strike', 'Change%', 'ITM', 'Profit $', '$/day', 'Profit %', 'Profit 1Y %', 'IV', 'OI', 'Delta', 'DualDelta', 'Gamma']
for j in range(0, min(distanceDate, len(ticker.options)-1)):
  out = [];
  opdtstr = ticker.options[j]
  calls = ticker.option_chain(opdtstr).calls
  inMndx1 = calls.loc[calls['inTheMoney'] == True].index
  if len(inMndx1)>0:
     inMndx = inMndx1[-1] 
  if inMndx < 1:
    continue
  opdt = datetime.strptime(opdtstr, date_format)
  print( f'\n   ========== Expiration date: {opdtstr} ==================', end='')
  for i in range(inMndx, min(len(calls)-1, (inMndx+distanceStrike-1))):
    dysLeft = (opdt-datetime.now()).days+1
    # if dysLeft < 2:
    if dysLeft < 2 or calls.iloc[i].inTheMoney:
      continue
    prc = calls.iloc[i].bid or calls.iloc[i].lastPrice or calls.iloc[i].ask
    earnPerDay = round(prc/dysLeft*100, 2)
    OI = round(calls.iloc[i].openInterest, 2)
    IV = round(calls.iloc[i].impliedVolatility*100, 2)
    # p = mibian.BS(  [tdPrc, calls.iloc[i].strike, interest_rate, dysLeft], putPrice=prc);
    bs = mibian.BS(  [tdPrc, calls.iloc[i].strike, interest_rate, dysLeft], volatility=IV);

    profitUSD = prc*100 - commissionUSD
    profit = profitUSD/tdPrc
    change = 0 if math.isnan(calls.iloc[i].percentChange) else round(calls.iloc[i].percentChange)
    toAdd = [ dysLeft, calls.iloc[i].strike, change, calls.iloc[i].inTheMoney, profitUSD, earnPerDay, profit, profit/dysLeft*365, IV, OI, bs.callDelta, bs.callDelta2, bs.gamma]
    out.append(toAdd)
    final.append(toAdd)
  display(pd.DataFrame(out, columns = colsArr ).round(2))

# openInterest	impliedVolatility
df = pd.DataFrame(final, columns = colsArr)
print()
printmd(f" - Prev.Day price {round(ytdPrc,2)}, Current price **{round(tdPrc,2)}**, diff: {round(diff,2)}%."+
        f" Final list ITM and OutOfMoney calls for nearest {distanceStrike} Strikes and {distanceDate} Dates")

# print('\n==== Total: =====')
data_table.DataTable(df.round(2), include_index=False, num_rows_per_page=200  ) # , min_width="500"


NameError: ignored

In [ ]:
#@title Historical volatility and variance { run: "auto" }
# ticker = yf.Ticker("TSLA")
yrwk=ticker.history(interval='1wk', period='52wk')['Close']
mo=ticker.history(interval='1d', period='4wk')['Close']
wk=ticker.history(interval='1d', period='1wk')['Close']
yr=ticker.history(interval='1d', period='52wk')['Close']
print('year  by day variance:', str(round(yr.std()/yr.median()*100,2))+'%',
      '; year  by day HV: ', round(yr.pct_change().std()*np.sqrt(365)*100,2),'%' )

print('year by week variance:', str(round(yrwk.std()/yrwk.median()*100,2))+'%',
      '; year by week HV: ', round(yrwk.pct_change().std()*np.sqrt(52)*100,2),'%' )

print('month by day variance:', str(round(mo.std()/mo.median()*100,2))+'%',
      '; month by day HV: ', round(mo.pct_change().std()*np.sqrt(30)*100,2),'%' )
print('week  by day variance:', str(round(wk.std()/wk.median()*100,2))+'%',
      '; week by day HV: ', round(wk.pct_change().std()*np.sqrt(7)*100,2),'%' )



In [ ]:
#@title Daily price movement distribution compared to normal { run: "auto" }
# ticker = yf.Ticker("VT")
import matplotlib.pyplot as plt
from scipy.stats import norm
plt.rcParams['figure.figsize'] = [16, 6]
a=ticker.history( period='250wk', interval='1d' )[['Close']]
df=a['Close'].pct_change()
df = df.dropna()
mean = np.mean(df)
std_dev = np.std(df)
df.hist(bins=80, histtype='stepfilled', alpha=0.5, density=True,  )
x = np.linspace( mean - 3*std_dev, mean + 3*std_dev, 100)
plt.plot(x, norm.pdf(x, mean, std_dev), "r")
plt.xlabel('Returns')
plt.ylabel('Fequency')
plt.show()

In [ ]:
#@title Daily price movements { run: "auto" }
# a.mean()
# plt.plot(a.diff())
plt.plot(a.pct_change())

In [ ]:
#@title Value at risk { run: "auto" }
VaR_90 = round(norm.ppf(1-0.9, mean, std_dev)*100,2)
VaR_95 = round(norm.ppf(1-0.95, mean, std_dev)*100,2)
VaR_99= round(norm.ppf(1-0.99, mean, std_dev)*100,2)
pd.DataFrame([['90%', str(VaR_90)+'%'], ['95%', str(VaR_95)+'%'], ["99%", str(VaR_99)+'%']], columns=['Confidence Level', 'Value at Risk'])


In [ ]:
#@title Price chart including 1,2 and 3 std diviations { run: "auto" }
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
# a=ticker.history( period='52wk', interval='1d' )[['High','Low']]

ticker2 = yf.Ticker('AAPL') 
# ticker2 = yf.Ticker('TSLA') 
a=ticker2.history( period='52wk', interval='1d' )[['Close']]

plt.title(" Dataset")
# plt.ylim(min(a['Close']) - 50, max(a['Close']) + 20)
plt.scatter(x=a.index, y=a['Close'])
# plt.scatter(x=a.index, y=a['Low'])
plt.axhline(y=a['Close'].mean(), xmin=0, xmax=1,  )
# plt.axhline(y=a['Low'].mean(), xmin=0, xmax=1, color='r'  )

plt.axhline(y=a['Close'].mean()+a['Close'].std(), xmin=0, xmax=1, ls='-.' )
# plt.axhline(y=a['Low'].mean()+a['Low'].std(), xmin=0, xmax=1, color='r', ls='-.'  )
plt.axhline(y=a['Close'].mean()-a['Close'].std(), xmin=0, xmax=1, ls='-.')
# plt.axhline(y=a['Low'].mean()-a['Low'].std(), xmin=0, xmax=1, color='r', ls='-.' )

plt.axhline(y=a['Close'].mean()+2*a['Close'].std(), xmin=0, xmax=1, ls='--', color='r')
# plt.axhline(y=a['Low'].mean()+2*a['Low'].std(), xmin=0, xmax=1, color='r', ls='--' )
plt.axhline(y=a['Close'].mean()-2*a['Close'].std(), xmin=0, xmax=1, ls='--', color='r'  )
# plt.axhline(y=a['Low'].mean()-2*a['Low'].std(), xmin=0, xmax=1, color='r', ls='--'  )

# plt.axhline(y=a['Close'].mean()+3*a['Close'].std(), xmin=0, xmax=1  )
# plt.axhline(y=a['Close'].mean()-3*a['Close'].std(), xmin=0, xmax=1, color='r' )

plt.show()



In [ ]:
#@title Print option chain { run: "auto" }
nearOdate=ticker.options[0]
date = "" 
 # @ param {type:"string"}
opt = ticker.option_chain(date or nearOdate)
# opts=opt[0]
# hf=int(len(opts)/2)
# inM=opts.loc[opts['inTheMoney'] ] # == True
# inMndx=opts.loc[opts['inTheMoney'] == True].index[0] 
# if inMndx<=distanceStrike:
  # distanceStrike = inMndx
# df1=opts.iloc[inMndx-distanceStrike:inMndx+distanceStrike].style.hide_index()
# df1.set_table_styles( [{'selector': 'tr:hover',   'props': [('background-color', 'yellow')]}] )
# opts.style.background_gradient()
# opts.round(2)
# pp=opts.round({"strike":1, "lastPrice":2, "bid":3, "ask":4})
# data_table.DataTable(opts[['strike','lastPrice','impliedVolatility','inTheMoney']].round(2), 
#                      include_index=False, num_rows_per_page=200  ) # , min_width="500"
print(f'Exp date: {nearOdate}')
print('Calls')
display(data_table.DataTable(opt[0].round(2), include_index=False, num_rows_per_page=200  )) # , min_width="500"
print(f'Exp date: {nearOdate}')
print('Puts')
display(data_table.DataTable(opt[1].round(2), include_index=False, num_rows_per_page=200  )) # , min_width="500"

# .style.hide_index().background_gradient(cmap='coolwarm', subset='impliedVolatility')
